# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff120d39d00>
├── 'a' --> tensor([[-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585]])
└── 'x' --> <FastTreeValue 0x7ff120d4efa0>
    └── 'c' --> tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                        [-0.3390, -0.1364, -0.5010, -0.6840],
                        [ 1.2164, -0.4171, -0.5446, -0.8840]])

In [4]:
t.a

tensor([[-1.0647,  0.1554, -0.0996],
        [-0.8743, -0.8145,  0.0585]])

In [5]:
%timeit t.a

70.9 ns ± 0.00928 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff120d39d00>
├── 'a' --> tensor([[ 0.4111, -2.2646,  0.5644],
│                   [ 0.3560,  2.9022,  0.0786]])
└── 'x' --> <FastTreeValue 0x7ff120d4efa0>
    └── 'c' --> tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                        [-0.3390, -0.1364, -0.5010, -0.6840],
                        [ 1.2164, -0.4171, -0.5446, -0.8840]])

In [7]:
%timeit t.a = new_value

69.2 ns ± 0.0621 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585]]),
    x: Batch(
           c: tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                      [-0.3390, -0.1364, -0.5010, -0.6840],
                      [ 1.2164, -0.4171, -0.5446, -0.8840]]),
       ),
)

In [10]:
b.a

tensor([[-1.0647,  0.1554, -0.0996],
        [-0.8743, -0.8145,  0.0585]])

In [11]:
%timeit b.a

58.6 ns ± 0.0947 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3528,  0.3572, -0.4879],
               [ 0.0138,  0.0475,  0.8865]]),
    x: Batch(
           c: tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                      [-0.3390, -0.1364, -0.5010, -0.6840],
                      [ 1.2164, -0.4171, -0.5446, -0.8840]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

804 ns ± 0.511 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 35 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff069f70850>
├── 'a' --> tensor([[[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]],
│           
│                   [[-1.0647,  0.1554, -0.0996],
│                    [-0.8743, -0.8145,  0.0585]]])
└── 'x' --> <FastTreeValue 0x7ff069f70880>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.7 µs ± 66.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff069f70610>
├── 'a' --> tensor([[-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585],
│                   [-1.0647,  0.1554, -0.0996],
│                   [-0.8743, -0.8145,  0.0585]])
└── 'x' --> <FastTreeValue 0x7ff069f93f40>
    └── 'c' --> tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                        [-0.3390, -0.1364, -0.5010, -0.6840],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]],
       
               [[-1.0647,  0.1554, -0.0996],
                [-0.8743, -0.8145,  0.0585]]]),
    x: Batch(
           c: tensor([[[-1.4610,  1.5212,  0.8402,  0.4299],
                       [-0.3390, -0.1364, -0.5010, -0.6840],
                       [ 1.2164, -0.4171, -0.5446, -0.8840]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585],
               [-1.0647,  0.1554, -0.0996],
               [-0.8743, -0.8145,  0.0585]]),
    x: Batch(
           c: tensor([[-1.4610,  1.5212,  0.8402,  0.4299],
                      [-0.3390, -0.1364, -0.5010, -0.6840],
                      [ 1.2164, -0.4171, -0.5446, -0.8840],
                      [-1.4610,  1.5212,  0.8402,  0.4299],
                      [-0.3390, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

343 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
